In [1]:
using CSV, DataFrames, Statistics, Gurobi, JuMP, Dates, CategoricalArrays

Preprocessing

In [2]:
data = CSV.read("KaggleV2-May-2016.csv", DataFrame)

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
,Float64,Int64,String1,String31,String31,Int64,String31,Int64,Int64,Int64,Int64,Int64,Int64,String3
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,No
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,Yes
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,Yes
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,No


In [3]:
#turn No-show into binary
data."No-show" = ifelse.(data."No-show" .== "No", 0, 1)

110527-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 1
 1
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [4]:
# 1. Parse the datetime strings (they have a trailing "Z", so strip it)
data.ScheduledDT   = DateTime.(replace.(data.ScheduledDay, "Z" => ""), dateformat"yyyy-mm-ddTHH:MM:SS")
data.AppointmentDT = DateTime.(replace.(data.AppointmentDay, "Z" => ""), dateformat"yyyy-mm-ddTHH:MM:SS")

# 2. Keep only the date part (drop time-of-day)
data.ScheduledDate   = Date.(data.ScheduledDT)
data.AppointmentDate = Date.(data.AppointmentDT)

# 3. Compute waiting time in days (scheduled - appointment)
data.lead_time = Dates.value.(data.AppointmentDate .- data.ScheduledDate)
data

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time
,Float64,Int64,String1,String31,String31,Int64,String31,Int64,Int64,Int64,Int64,Int64,Int64,Int64,DateTime,DateTime,Date,Date,Int64
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,0,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0


In [5]:
#filter ages between 0 and 100
data = filter(row -> (row.Age >= 0) && (row.Age <= 100), data)

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time
,Float64,Int64,String1,String31,String31,Int64,String31,Int64,Int64,Int64,Int64,Int64,Int64,Int64,DateTime,DateTime,Date,Date,Int64
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,0,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0


In [6]:
#make neighborhood and gender a categorical variable
data.Neighbourhood = categorical(data.Neighbourhood)
data.Gender = categorical(data.Gender)
data.Scholarship = categorical(data.Scholarship)
data.Hipertension = categorical(data.Hipertension)
data.Diabetes = categorical(data.Diabetes)
data.Alcoholism = categorical(data.Alcoholism)
data.Handcap = categorical(data.Handcap)
data.SMS_received = categorical(data.SMS_received)
data

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time
,Float64,Int64,Cat…,String31,String31,Int64,Cat…,Cat…,Cat…,Cat…,Cat…,Cat…,Cat…,Int64,DateTime,DateTime,Date,Date,Int64
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,0,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0


In [7]:
#drop duplicate rows
data = unique(data)

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time
,Float64,Int64,Cat…,String31,String31,Int64,Cat…,Cat…,Cat…,Cat…,Cat…,Cat…,Cat…,Int64,DateTime,DateTime,Date,Date,Int64
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,0,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0


In [8]:
#create day of week and appointment month features
data.day_of_week = dayname.(data.AppointmentDate)
data.appointment_month = monthname.(data.AppointmentDate)

#turn day of week and appointment month into categorical variables
data.day_of_week = categorical(data.day_of_week, ordered=true, levels=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
data.appointment_month = categorical(data.appointment_month, ordered=true, levels=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"])
data

Row,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDT,AppointmentDT,ScheduledDate,AppointmentDate,lead_time,day_of_week,appointment_month
,Float64,Int64,Cat…,String31,String31,Int64,Cat…,Cat…,Cat…,Cat…,Cat…,Cat…,Cat…,Int64,DateTime,DateTime,Date,Date,Int64,Cat…,Cat…
1,2.98725e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2016-04-29T18:38:08,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
2,5.58998e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2016-04-29T16:08:27,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
3,4.26296e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,0,2016-04-29T16:19:04,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
4,8.67951e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2016-04-29T17:29:31,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
5,8.84119e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,0,2016-04-29T16:07:23,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April
6,9.59851e13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,0,2016-04-27T08:36:51,2016-04-29T00:00:00,2016-04-27,2016-04-29,2,Friday,April
7,7.33688e14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:05:12,2016-04-29T00:00:00,2016-04-27,2016-04-29,2,Friday,April
8,3.44983e12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,1,2016-04-27T15:39:58,2016-04-29T00:00:00,2016-04-27,2016-04-29,2,Friday,April
9,5.63947e13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,0,2016-04-29T08:02:16,2016-04-29T00:00:00,2016-04-29,2016-04-29,0,Friday,April


In [9]:
y = data."No-show"
# X = select(data, [:Gender, :Age, :Neighbourhood, :Scholarship, :Hipertension, :Diabetes, :Alcoholism, :Handcap, :SMS_received, :lead_time, :day_of_week, :appointment_month])
X = select(data, [:Gender, :Age, :Scholarship, :Hipertension, :Diabetes, :Alcoholism, :Handcap, :SMS_received, :lead_time, :day_of_week, :appointment_month])


Row,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,lead_time,day_of_week,appointment_month
,Cat…,Int64,Cat…,Cat…,Cat…,Cat…,Cat…,Cat…,Int64,Cat…,Cat…
1,F,62,0,1,0,0,0,0,0,Friday,April
2,M,56,0,0,0,0,0,0,0,Friday,April
3,F,62,0,0,0,0,0,0,0,Friday,April
4,F,8,0,0,0,0,0,0,0,Friday,April
5,F,56,0,1,1,0,0,0,0,Friday,April
6,F,76,0,1,0,0,0,0,2,Friday,April
7,F,23,0,0,0,0,0,0,2,Friday,April
8,F,39,0,0,0,0,0,0,2,Friday,April
9,F,21,0,0,0,0,0,0,0,Friday,April


In [10]:
#split test and train sets
seed = 15095
(X_trainvalid, y_trainvalid), (X_test, y_test) = IAI.split_data(:classification, X, y, seed=seed, train_proportion=0.8)
(X_train, y_train), (X_valid, y_valid) =  IAI.split_data(:classification, X_trainvalid, y_trainvalid, seed=seed, train_proportion=0.8);

RF

In [ ]:
# TODO Train a Random Forest 
# Grid search for hyperparameter tuning
max_depths = [15]
num_trees = [200]
minbuckets = [100, 200]
rf_grid = IAI.GridSearch(
    IAI.RandomForestClassifier(
        random_seed=seed,
        max_categoric_levels_before_warning=10000,
    ),
    max_depth=max_depths,
    num_trees=num_trees,
    minbucket=minbuckets,
)
# Fit training data
IAI.fit_cv!(rf_grid, X_trainvalid, Array(y_trainvalid), validation_criterion=:auc, n_folds=3)


# Calculate AUC score on the validation set (name the variable holding the AUC score as `val_auc`)
val_auc = IAI.score(rf_grid, X_valid, y_valid, criterion=:auc)

┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ nothing nothing:nothing
┌ Warning: Interpretable AI license expires soon: 2025-12-31T00:00:00. If you need to renew, please send us the following machine ID:
│ 1a2559dd14033a19804c4627f90f114af9c4fbd0d9ee19e3281af640f42d83a2
└ @ nothing nothing:nothing


In [ ]:
# TODO: Get best model
model = rf_grid
lnr = IAI.get_learner(model)

# TODO: Display selected model's parameters
params = IAI.get_params(lnr)
println(params)
println("Max depth of the best model is ", params[:max_depth])
println("Number of trees of the best model is ", params[:num_trees])
println("Minbucket size of the best model is ", params[:minbucket])

In [ ]:
# TODO: Calculate AUC score for test data
auc_score_rf = IAI.score(model, X_test, y_test, criterion=:auc)

# Get class predictions for test set
acc_rf = IAI.score(model, X_test, y_test, criterion=:misclassification)

println("Test AUC score: ", round(auc_score_rf, digits=4))
println("Test Accuracy: ", round(acc_rf, digits=4))

CART

In [ ]:

max_depths = [8,10,12]
minbuckets = [100,200]

grid_cart = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        random_seed=seed,
        localsearch = false,
    ),
    max_depth=max_depths,
    minbucket=minbuckets,
    criterion=:gini
)

IAI.fit_cv!(grid_cart, X_trainvalid, Array(y_trainvalid), validation_criterion=:auc, n_folds=3)

In [ ]:
# TODO: Get best model
model = grid_cart
lnr = IAI.get_learner(model)

# TODO: Display selected model's parameters
params = IAI.get_params(lnr)
println(params)
println("Max depth of the best model is ", params[:max_depth])
println("Minbucket size of the best model is ", params[:minbucket])

In [ ]:
# TODO: Calculate AUC score for test data
auc_score_cart = IAI.score(model, X_test, y_test, criterion=:auc)

# Get class predictions for test set
acc_cart = IAI.score(model, X_test, y_test, criterion=:misclassification)

println("Test AUC score: ", round(auc_score_cart, digits=4))
println("Test Accuracy: ", round(acc_cart, digits=4))

OCT

In [ ]:
# Train an OCT, this grid search might take some time
# Grid search for hyperparameter tuning
# TODO: 
max_depths = [4,6,8]
minbuckets = [100, 200]

grid_oct = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        random_seed=seed,
        max_categoric_levels_before_warning=10000,
    ),
    max_depth=max_depths,
    minbucket=minbuckets,
    criterion=:gini,
) 

IAI.fit_cv!(grid_oct, X_trainvalid, Array(y_trainvalid), validation_criterion=:auc, n_folds=3)

In [ ]:
# TODO: Get best model
model = grid_oct
lnr = IAI.get_learner(model)

# TODO: Display selected model's parameters
params = IAI.get_params(lnr)
println(params)
println("Max depth of the best model is ", params[:max_depth])
println("Minbucket size of the best model is ", params[:minbucket])

In [ ]:
# TODO: Calculate AUC score for test data
auc_score_oct = IAI.score(model, X_test, y_test, criterion=:auc)

# Get class predictions for test set
acc_oct = IAI.score(model, X_test, y_test, criterion=:misclassification)

println("Test AUC score: ", round(auc_score_oct, digits=4))
println("Test Accuracy: ", round(acc_oct, digits=4))